In [4]:
%load_ext autoreload
%autoreload 2

from shared import Db, Fn

from main.ai import ai_setups
from main.data import Mol, data
from main.tree import JsonTree, Tree, TreeTypes
from main.types import AiInput, Setup

from IPython.display import display, HTML

import numpy as np
import pandas
from main.utils import fn_txt

def get_stats(mol: Mol, setup: Setup):
    ai_input: AiInput = {"smiles": mol.smiles, "setup": setup}
    json_tree = db.read(["ai_postprocess", ai_input], JsonTree)
    assert json_tree
    tree = Tree(json_tree)
    return tree.type, tree.stats()

filters: list[Fn[TreeTypes, bool]] = [
    lambda _type: True,
    lambda type: type == "internal",
    lambda type: type == "not_solved"
]

full_names = {"sa": "SAscore", "sc": "SCScore", "ra": "RAscore",
              "syba": "SYBA"}
agg_names = {"min": "minimum", "max": "maximum", "avg": "mean"}
    
with Db("db_modified", True) as db:
    mols = data()
    setup_and_stats = [(setup, [get_stats(mol, setup) for mol in mols]) for setup in ai_setups]
    for filter in filters:
        display(HTML("============================================="))
        display(HTML("filter: " + fn_txt(filter)))
        for characteristic in ("max_depth", "max_width", "node_count", "not_solved_count"):
            display(HTML(characteristic))
            rows: list[tuple[str,float,float,float]] = []
            for s, trees in setup_and_stats:
                l: list[int] = [stats[characteristic] for (type, stats) in trees if filter(type)]
                setup = f"{full_names[s['score']]} {agg_names[s['agg']]}"
                rows.append((setup, s['uw_multiplier'], np.std(l).item(), np.average(l).item(), np.median(l).item()))
            display(pandas.DataFrame(rows, columns=["Score", "Exchanged score fraction", "Std deviation", "Mean", "Median"]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,2.966002,8.755102,9.0
1,SCScore minimum,0.2375,2.984759,8.775510,9.0
2,SCScore maximum,0.2375,3.016959,8.857143,9.0
3,SCScore minimum,0.4750,3.114369,8.877551,9.0
4,SCScore maximum,0.4750,2.948397,8.795918,9.0
5,SCScore minimum,0.7125,3.016821,8.795918,9.0
6,SCScore maximum,0.7125,3.026056,8.836735,9.0
7,SAscore minimum,0.2375,3.030320,8.795918,9.0
8,SAscore maximum,0.2375,2.950374,8.775510,9.0
9,SAscore minimum,0.4750,3.079537,8.836735,9.0


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,79.605493,115.836735,104.0
1,SCScore minimum,0.2375,72.193571,112.918367,102.0
2,SCScore maximum,0.2375,92.332363,121.000000,106.0
3,SCScore minimum,0.4750,70.729288,115.979592,108.0
4,SCScore maximum,0.4750,79.498344,119.183673,108.0
5,SCScore minimum,0.7125,78.982919,119.326531,109.0
6,SCScore maximum,0.7125,71.927795,116.673469,109.0
7,SAscore minimum,0.2375,77.377965,118.448980,108.0
8,SAscore maximum,0.2375,80.494308,119.183673,106.0
9,SAscore minimum,0.4750,78.892469,119.244898,109.0


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,383.891782,624.448980,622.0
1,SCScore minimum,0.2375,357.194631,614.530612,616.0
2,SCScore maximum,0.2375,385.142319,641.918367,646.0
3,SCScore minimum,0.4750,362.632884,640.408163,650.0
4,SCScore maximum,0.4750,373.379975,645.081633,649.0
5,SCScore minimum,0.7125,360.175180,644.408163,649.0
6,SCScore maximum,0.7125,368.898435,645.836735,650.0
7,SAscore minimum,0.2375,373.173272,639.510204,645.0
8,SAscore maximum,0.2375,385.174569,644.979592,649.0
9,SAscore minimum,0.4750,372.952424,643.571429,650.0


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,286.580406,223.551020,132.0
1,SCScore minimum,0.2375,244.677409,211.836735,134.0
2,SCScore maximum,0.2375,295.157197,230.326531,136.0
3,SCScore minimum,0.4750,260.086613,223.122449,136.0
4,SCScore maximum,0.4750,279.672345,229.306122,138.0
5,SCScore minimum,0.7125,263.093437,225.081633,140.0
6,SCScore maximum,0.7125,265.976406,227.020408,141.0
7,SAscore minimum,0.2375,268.964581,225.183673,137.0
8,SAscore maximum,0.2375,287.977563,230.673469,136.0
9,SAscore minimum,0.4750,273.662586,227.183673,138.0


...

,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,135.231551,110.863636,93.0
1,SCScore minimum,0.2375,133.100200,115.136364,98.5
2,SCScore maximum,0.2375,136.961236,116.272727,97.0
3,SCScore minimum,0.4750,133.207390,120.136364,106.5
4,SCScore maximum,0.4750,137.836111,120.454545,105.5
5,SCScore minimum,0.7125,131.518319,121.500000,107.5
6,SCScore maximum,0.7125,134.953405,121.818182,101.5
7,SAscore minimum,0.2375,138.614454,117.818182,98.5
8,SAscore maximum,0.2375,134.886921,116.136364,96.5
9,SAscore minimum,0.4750,137.037728,120.454545,104.5


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,2.981424,9.333333,9.0
1,SCScore minimum,0.2375,3.059089,9.444444,9.0
2,SCScore maximum,0.2375,3.047408,9.481481,9.0
3,SCScore minimum,0.4750,3.213056,9.518519,9.0
4,SCScore maximum,0.4750,2.945784,9.370370,9.0
5,SCScore minimum,0.7125,3.059089,9.444444,9.0
6,SCScore maximum,0.7125,3.071172,9.444444,9.0
7,SAscore minimum,0.2375,3.059089,9.444444,9.0
8,SAscore maximum,0.2375,2.945784,9.370370,9.0
9,SAscore minimum,0.4750,3.166179,9.444444,9.0


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,88.801871,141.925926,113.0
1,SCScore minimum,0.2375,79.971823,133.629630,110.0
2,SCScore maximum,0.2375,108.158447,148.481481,111.0
3,SCScore minimum,0.4750,77.077322,137.888889,114.0
4,SCScore maximum,0.4750,89.503155,144.000000,114.0
5,SCScore minimum,0.7125,89.286487,143.814815,110.0
6,SCScore maximum,0.7125,78.710875,139.074074,114.0
7,SAscore minimum,0.2375,86.053960,142.888889,115.0
8,SAscore maximum,0.2375,90.328618,145.000000,111.0
9,SAscore minimum,0.4750,88.666388,143.888889,111.0


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,381.143212,770.000000,675.0
1,SCScore minimum,0.2375,352.336415,735.296296,675.0
2,SCScore maximum,0.2375,385.157563,782.851852,677.0
3,SCScore minimum,0.4750,357.384018,770.185185,676.0
4,SCScore maximum,0.4750,369.779440,780.407407,680.0
5,SCScore minimum,0.7125,356.148895,771.962963,672.0
6,SCScore maximum,0.7125,366.542250,776.962963,676.0
7,SAscore minimum,0.2375,364.714075,777.703704,675.0
8,SAscore maximum,0.2375,382.741759,788.925926,676.0
9,SAscore minimum,0.4750,370.357244,777.259259,675.0


,Score,Exchanged score fraction,Std deviation,Mean,Median
0,SCScore maximum,0.0000,339.660195,315.370370,177.0
1,SCScore minimum,0.2375,283.522440,290.629630,168.0
2,SCScore maximum,0.2375,351.542589,323.259259,177.0
3,SCScore minimum,0.4750,304.337469,307.037037,177.0
4,SCScore maximum,0.4750,330.071373,318.000000,176.0
5,SCScore minimum,0.7125,309.287278,309.481481,174.0
6,SCScore maximum,0.7125,311.737971,312.740741,180.0
7,SAscore minimum,0.2375,313.983250,312.666667,180.0
8,SAscore maximum,0.2375,340.998860,324.000000,179.0
9,SAscore minimum,0.4750,322.129684,314.148148,179.0
